In [ ]:
import time
import smbus2
import bme280

BME280_I2C_ADDR = 0x76
I2C_BUS = 1

bus = smbus2.SMBus(I2C_BUS)

calibration_params = bme280.load_calibration_params(bus, BME280_I2C_ADDR)

def read_sensor_data():
        data = bme280.sample(bus,BME280_I2C_ADDR, calibration_params)

        temperature = data.temperature
        

        return temperature

if __name__ == "__main__":
        while True:
            temp = read_sensor_data()
            print(f"Temperature: {temp:.2f}C")
            time.sleep(30) # read data every minute

Temperature: 32.01C
Temperature: 32.32C
Temperature: 32.43C
Temperature: 32.49C
Temperature: 32.02C
Temperature: 31.80C
Temperature: 31.74C
Temperature: 31.51C
Temperature: 31.55C
Temperature: 31.48C
Temperature: 31.34C
Temperature: 31.54C
Temperature: 31.32C
Temperature: 31.52C
Temperature: 31.70C
Temperature: 31.74C
